In [17]:
!pip install ta

In [18]:
# Required Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout
from tensorflow.keras.optimizers import Adam
import ta

In [20]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).
/gdrive


In [5]:
# Load Data
# !ls '/gdrive/MyDrive/Trading Setup'
nifty50 = pd.read_csv('/gdrive/MyDrive/Trading Setup/nifty-50-minute-data.csv', index_col='date', parse_dates=True)
print(nifty50.head())

                        open     high      low    close
date                                                   
2015-01-09 09:15:00  8285.45  8295.90  8285.45  8292.10
2015-01-09 09:16:00  8292.60  8293.60  8287.20  8288.15
2015-01-09 09:17:00  8287.40  8293.90  8287.40  8293.90
2015-01-09 09:18:00  8294.25  8300.65  8293.90  8300.65
2015-01-09 09:19:00  8300.60  8301.30  8298.75  8301.20


In [6]:
# Feature Engineering with Technical Indicators
# Define features (calculate intraday indicators)
nifty50['MA10'] = ta.trend.sma_indicator(nifty50['close'], window=10)
nifty50['EMA10'] = ta.trend.ema_indicator(nifty50['close'], window=10)

# Moving Averages
nifty50['MA50'] = nifty50['close'].rolling(window=50).mean()
nifty50['MA200'] = nifty50['close'].rolling(window=200).mean()

# Exponential Moving Averages
nifty50['EMA20'] = nifty50['close'].ewm(span=20, adjust=False).mean()
nifty50['EMA50'] = nifty50['close'].ewm(span=50, adjust=False).mean()
nifty50['EMA200'] = nifty50['close'].ewm(span=200, adjust=False).mean()

# Relative Strength Index (RSI)
nifty50['RSI'] = ta.momentum.rsi(nifty50['close'], window=14)

# Moving Average Convergence Divergence (MACD)
nifty50['MACD'] = ta.trend.macd(nifty50['close'])
nifty50['MACD_Signal'] = ta.trend.macd_signal(nifty50['close'])
nifty50['MACD_Diff'] = ta.trend.macd_diff(nifty50['close'])

# Bollinger Bands
bb = ta.volatility.BollingerBands(close=nifty50['close'], window=20, window_dev=2)
nifty50['BB_High'] = bb.bollinger_hband()
nifty50['BB_Low'] = bb.bollinger_lband()
nifty50['BB_Mid'] = bb.bollinger_mavg()

# Stochastic Oscillator
stoch = ta.momentum.StochasticOscillator(high=nifty50['high'], low=nifty50['low'], close=nifty50['close'], window=14, smooth_window=3)
nifty50['Stoch_%K'] = stoch.stoch()
nifty50['Stoch_%D'] = stoch.stoch_signal()

# Average True Range (ATR)
nifty50['ATR'] = ta.volatility.average_true_range(high=nifty50['high'], low=nifty50['low'], close=nifty50['close'], window=14)

# On-Balance Volume (OBV)
# nifty50['OBV'] = ta.volume.on_balance_volume(nifty50['close'], nifty50['volume'])

# Pivot Points, Supports, and Resistances
nifty50['Pivot'] = (nifty50['high'] + nifty50['low'] + nifty50['close']) / 3
nifty50['S1'] = 2 * nifty50['Pivot'] - nifty50['high']
nifty50['R1'] = 2 * nifty50['Pivot'] - nifty50['low']
nifty50['S2'] = nifty50['Pivot'] - (nifty50['high'] - nifty50['low'])
nifty50['R2'] = nifty50['Pivot'] + (nifty50['high'] - nifty50['low'])

# Historical Highs and Lows (using rolling windows)
nifty50['Rolling_High'] = nifty50['high'].rolling(window=20).max()
nifty50['Rolling_Low'] = nifty50['low'].rolling(window=20).min()

# Fill NaN values
nifty50.fillna(method='bfill', inplace=True)

# # Create Buy/Sell Signal (1 for Buy, -1 for Sell, 0 for Hold)
# nifty50['Signal'] = 0
# nifty50['Signal'][nifty50['close'] > nifty50['BB_High']] = -1
# nifty50['Signal'][nifty50['close'] < nifty50['BB_Low']] = 1



<ipython-input-6-63a4e30b6325>:52: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  nifty50.fillna(method='bfill', inplace=True)


In [7]:
def create_sliding_window(data, window_size=30):
    X = []
    y = []
    for i in range(window_size, len(data)):
        X.append(data[i-window_size:i])  # Take `window_size` number of past data points
        y.append(data[i])  # Predict the next data point (or target)
    return np.array(X), np.array(y)

# Creating input windows for the model
window_size =1000
features = nifty50[['close', 'RSI', 'EMA20','EMA50', 'EMA200', 'BB_High', 'BB_Low','MACD',
                    'MACD_Signal', 'MACD_Diff', 'Pivot', 'S1', 'R1', 'S2', 'R2']].dropna()
X, y = create_sliding_window(features.values, window_size)

# Display the shape of the created windows
X.shape, y.shape


((835757, 1000, 15), (835757, 15))

In [8]:
# Split Data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)



In [9]:
# Standardize Data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train.reshape(-1, X_train.shape[-1])).reshape(X_train.shape)
X_test = scaler.transform(X_test.reshape(-1, X_test.shape[-1])).reshape(X_test.shape)


In [10]:

# Build LSTM Model for Reinforcement Learning
model = Sequential()
model.add(LSTM(50, input_shape=(X_train.shape[1], X_train.shape[2]), return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(50))
model.add(Dropout(0.2))
model.add(Dense(1, activation='tanh'))



In [12]:
# Compile Model
model.compile(optimizer=Adam(learning_rate=0.001), loss='mse')

Save Model and x,y data

In [13]:
# Save the compiled model to a file
model.save('/gdrive/MyDrive/Trading Setup/lstm_model.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [1]:
print("abg")

abg


In [ ]:
import numpy as np

# Save the data to .npy files
np.save('/gdrive/MyDrive/Trading Setup/X_train.npy', X_train)
# np.save('/gdrive/MyDrive/Trading Setup/X_test.npy', X_test)
# np.save('/gdrive/MyDrive/Trading Setup/y_train.npy', y_train)
# np.save('/gdrive/MyDrive/Trading Setup/y_test.npy', y_test)

Now load the Model and x,y data from gdrive

In [15]:
# Load the model from Gdrive for further training or inference
from tensorflow.keras.models import load_model
model = load_model('/gdrive/MyDrive/Trading Setup/lstm_model.h5')

In [16]:
# To load from Gdrive
X_train = np.load('X_train.npy')
X_test = np.load('X_test.npy')
y_train = np.load('y_train.npy')
y_test = np.load('y_test.npy')

FileNotFoundError: [Errno 2] No such file or directory: 'X_train.npy'

Run below code with High GPU  and Above code with High Ram

In [11]:
history = model.fit(X_train, y_train, epochs=50, batch_size=16, validation_split=0.1, verbose=1)

Epoch 1/50
 1069/37609 [..............................] - ETA: 9:43:19 - loss: 125409872.0000

KeyboardInterrupt: 

In [ ]:
# Predict and Evaluate
y_pred = model.predict(X_test)
y_pred = np.where(y_pred > 0, 1, np.where(y_pred < 0, -1, 0))

accuracy = accuracy_score(y_test, y_pred)
print(f'Model Accuracy: {accuracy * 100:.2f}%')


In [ ]:

# Plot Model Loss
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Model Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()



In [ ]:
# Generate Buy/Sell signals on the entire dataset
nifty50['Predicted_Signal'] = np.nan
nifty50['Predicted_Signal'][-len(y_pred):] = y_pred



In [ ]:
# Plot Buy/Sell signals
plt.figure(figsize=(14, 7))
plt.plot(nifty50['close'], label='Close Price', alpha=0.5)
plt.plot(nifty50['Predicted_Signal'].replace(0, np.nan), 'ro', markersize=3, label='Signal')
plt.title('Buy/Sell Signals')
plt.xlabel('Datetime')
plt.ylabel('Price')
plt.legend()
plt.show()